In [ ]:
from json import loads
import pandas as pd
import numpy as np

In [ ]:
song_scraped= loads(open("artists_scraped.json").read())
BB = pd.read_csv("data/Hot Stuff.csv")

In [ ]:
print("No. Total Artists:    ", len(set(BB["Performer"])))
print("No. Artists Scraped:  ", len(song_scraped))
print("No. Missing Artists:  ", len(set(BB["Performer"]) - set(song_scraped.keys())))

### Examine Commonality Between The Missing Artists

In [ ]:
from collections import Counter

In [ ]:
missing_artists = list(set(BB["Performer"]) - set(song_scraped.keys()))
def get_count(ls):
    bag_of_artists_name = list()
    for i in ls:
        bag_of_artists_name += i.split(" ")
    name_counts = Counter(bag_of_artists_name)
    print(name_counts.most_common(20))
get_count(missing_artists)

In [ ]:
missing_artists_reformat = list(missing_artists)
for i in range(len(missing_artists)):
    if "Featuring" in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("Featuring")[0].strip()
    if "Feat." in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("Feat.")[0].strip()
    if "featuring" in missing_artists[i]:
        missing_artists_reformat[i] = missing_artists[i].split("featuring")[0].strip()
    if "&" in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("&")[0].strip()
    if "and" in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("and")[0].strip()
    if "And" in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("And")[0].strip()
    if "With" in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("With")[0].strip()
    if "with" in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("with")[0].strip()
    if "(" in missing_artists_reformat[i]:
        missing_artists_reformat[i] = missing_artists[i].split("(")[0].strip()
get_count(missing_artists_reformat)

In [ ]:
import requests
import re
from random import randrange
from bs4 import BeautifulSoup
from collections import defaultdict

In [ ]:
from multiprocessing import Pool, TimeoutError


art_song_links = dict()
comm = r'<table style="padding:2px; border-spacing:0px; border:0px; width:100%">\n<tr>\n<td style="width:50%; vertical-align:top;"><b class="text-yellow">Song<\/b><\/td>\n<td style="width:50%; vertical-align:top;"><b class="text-yellow">in Album<\/b><\/td><td>&nbsp;<\/td>\n<td>&nbsp;<\/td>\n<\/tr>\n([\S\s]*)\n<\/table>'
missing_artists_2 = set()
def scrape_data(i):
    ex = missing_artists_reformat[i]
    songs_array = []; album_array = []; url_array = []
    url = "https://lyrics.az/{}/allsongs.html"
    resp = requests.get(url.format("-".join(ex.lower().split())))
    m = re.search(comm, resp.text)
    if m:
        rows = m.group(1)
        soup = BeautifulSoup(rows, 'html.parser')
        for r in soup.find_all('tr'):
            find_link_album_song = r'<tr>\n<td style=\"vertical-align:top;\"><a href=\"(.*\.html)\"><div class=\"schemadiv\" itemscope=\"\" itemtype=\"http:\/\/schema\.org\/MusicRecording\">\n<span itemprop=\"name\">(.*)<\/span><\/div><\/a><\/td>\n<td style=\"vertical-align:top;\">\[(<a href=\".*\" style=\"color: #ffffff;\">(.*)<\/a>)?]<\/td>\n<td>\s<\/td>\n<td>\s<\/td>\n<\/tr>'
            #print("'{}'".format(str(r)))
            m = re.search(find_link_album_song, str(r))
            #print(m.group(1),m.group(2),m.group(4))
            songs_array.append(m.group(2))
            album_array.append(m.group(4))
            url_array.append("https://lyrics.az{}".format(m.group(1)))
        art_song_links[missing_artists[i]] = {"Song": songs_array, "Album": album_array, "Url": url_array}
        print("ind {}".format(i))
        return "index {} found :)".format(i)
    else:
        print(ex)
        missing_artists_2.add(missing_artists[i])
        return "index {} not found :(".format(i)
    
comm = '''
with Pool(processes=2) as pool:
    try:
        for res in pool.imap_unordered(scrape_data, range(len(missing_artists))):
            #print(res)
            pass
    except:
        print('Stopped search at index {}'.format(i))
'''

In [10]:
len(missing_artists_2)

2124

In [12]:
from json import dumps

In [13]:
jdata = dumps(art_song_links)
output_file = open("stripped_art_data.json","w+")
output_file.write(jdata)
output_file.close()

In [19]:
art_song = defaultdict(set)
for i,row in BB.iterrows():
    art_song[row['Performer']].add(row['Song'])
len(art_song)

9447

In [15]:
missing_songs = defaultdict(set) # key is artist, value is set of songs
for art in art_song:
    if 


,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/2012-1...,12/15/2012,21,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,28.0,21,11
1,http://www.billboard.com/charts/hot-100/2012-1...,12/22/2012,14,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,21.0,14,12
2,http://www.billboard.com/charts/hot-100/2012-1...,12/29/2012,12,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,14.0,12,13
3,http://www.billboard.com/charts/hot-100/2013-0...,1/5/2013,11,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,12.0,11,14
4,http://www.billboard.com/charts/hot-100/2013-0...,1/12/2013,15,Don't You Worry Child,Swedish House Mafia Featuring John Martin,Don't You Worry ChildSwedish House Mafia Featu...,1,11.0,11,15


In [20]:
art_song_links.keys()

dict_keys(['Kid Cudi', 'Drake', 'KYLE', 'Cheat Codes', 'Twista', 'Ill Al Skratch', 'OutKast', 'Jellybean', 'Jamie Foxx', 'Kenny Chesney', 'Shakira', 'Christina Aguilera', 'Migos', 'Taylor Swift', 'Rod Stewart', 'Sonny', 'Tory Lanez', 'Elvin Bishop', 'The Brooklyn Bridge', 'Eminem', 'Gucci Mane', 'Bubba Sparxxx', 'Tony Yayo', 'Post Malone', 'Jimmy Davis', 'Carrie Underwood', 'J-Shin', 'Les Cooper', 'John Travolta', 'Pitbull', '50 Cent', 'Sylvia', 'Justin Bieber', 'Metro Boomin', 'Kevin Rudolf', 'Dick Dale', 'Travie McCoy', 'Tyga', 'Rodney O', 'BlocBoy JB', 'Jhene Aiko', 'MAX', 'Diana Ross', 'Grace', 'Trick Daddy', 'Fifth Harmony', 'Skrillex', 'Dolly Parton', 'George Thorogood', 'Kenny Rogers', 'Jay-Z', 'JAY-Z', 'David Guetta', 'Dionne Warwick', 'Demi Lovato', 'Alan Jackson', 'Icona Pop', 'Kane Brown', 'Havana Brown', 'Meek Mill', 'Busta Rhymes', 'Katy Perry', 'Ferrante', 'Nicole', 'Calvin Harris', 'Jeanette', 'Shawn Mendes', 'Lupe Fiasco', 'Julio Iglesias', 'Major Lazer', 'Estelle', 'Th

In [38]:
missing_artists

['Kid Cudi Featuring MGMT & Ratatat',
 'Voices Of America',
 'Drake Featuring Quavo & Travis Scott',
 'RTZ',
 'Kirby St. Romain',
 'KYLE Featuring Lil Yachty',
 'Drake Featuring Nicki Minaj',
 'Cheat Codes Featuring Demi Lovato',
 'Jazzy Jeff & The Fresh Prince',
 'Twista Featuring Kanye West & Jamie Foxx',
 'The Village Soul Choir',
 'Ill Al Skratch Featuring Brian McKnight',
 'Fruit De La Passion',
 'OutKast Featuring Killer Mike',
 'Jellybean Featuring Elisa Fiorillo',
 'What Is This',
 'Jamie Foxx Featuring Chris Brown',
 'Kenny Chesney With Dave Matthews',
 'Shakira Featuring Nicky Jam',
 'Christina Aguilera Featuring Missy Elliott',
 'Migos Featuring 21 Savage',
 'Taylor Swift Featuring Ed Sheeran & Future',
 'Rod Stewart With Ronald Isley',
 'Sonny & Cher',
 'Tory Lanez & Rich The Kid',
 'Elvin Bishop Featuring Mickey Thomas',
 'Johnnie & Joe',
 'The Brooklyn Bridge Featuring Johnny Maestro',
 'Wayne Massey',
 'Eminem Featuring Dr. Dre',
 'Don Shirley',
 'Gucci Mane Featuring Ka

In [39]:
missing_artists_reformat

['Kid Cudi',
 'Voices Of America',
 'Drake',
 'RTZ',
 'Kirby St. Romain',
 'KYLE',
 'Drake',
 'Cheat Codes',
 'Jazzy Jeff',
 'Twista',
 'The Village Soul Choir',
 'Ill Al Skratch',
 'Fruit De La Passion',
 'OutKast',
 'Jellybean',
 'What Is This',
 'Jamie Foxx',
 'Kenny Chesney',
 'Shakira',
 'Christina Aguilera',
 'Migos',
 'Taylor Swift',
 'Rod Stewart',
 'Sonny',
 'Tory Lanez',
 'Elvin Bishop',
 'Johnnie',
 'The Brooklyn Bridge',
 'Wayne Massey',
 'Eminem',
 'Don Shirley',
 'Gucci Mane',
 'Bubba Sparxxx',
 'Poni-Tails',
 'Jay Z Kanye West',
 'Tony Yayo',
 'Post Malone',
 'Jimmy Davis',
 'The James Boys',
 'Alvin Cash',
 'Carrie Underwood',
 'Lois Fletcher',
 'J-Shin',
 'Les Cooper',
 'John Travolta',
 'Grover Washington, Jr.',
 'Pitbull',
 '50 Cent',
 'Sylvia',
 "Herb Alpert's Tijuana Brass",
 'Lena Zavaroni',
 '2 Of Clubs',
 'Vanessa Williams/Brian McKnight',
 'Justin Bieber',
 'Metro Boomin',
 '"Pookie" Hudson',
 'Kevin Rudolf',
 'Dick Dale',
 'Bob McFadden',
 'Kimberly Scott',
 "